**Import Packages**

In [2]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras as tf
from keras.layers import Dense, Activation
from keras import Sequential
from keras.models import load_model as tl
from keras.optimizers import Adam

**Read dataset**

In [3]:
data = pd.read_csv("/content/drive/MyDrive/Dataset_CKD.csv")
print(data)

      id   age    bp     sg   al   su     rbc        pc         pcc  \
0      0  48.0  80.0  1.020  1.0  0.0     NaN    normal  notpresent   
1      1   7.0  50.0  1.020  4.0  0.0     NaN    normal  notpresent   
2      2  62.0  80.0  1.010  2.0  3.0  normal    normal  notpresent   
3      3  48.0  70.0  1.005  4.0  0.0  normal  abnormal     present   
4      4  51.0  80.0  1.010  2.0  0.0  normal    normal  notpresent   
..   ...   ...   ...    ...  ...  ...     ...       ...         ...   
395  395  55.0  80.0  1.020  0.0  0.0  normal    normal  notpresent   
396  396  42.0  70.0  1.025  0.0  0.0  normal    normal  notpresent   
397  397  12.0  80.0  1.020  0.0  0.0  normal    normal  notpresent   
398  398  17.0  60.0  1.025  0.0  0.0  normal    normal  notpresent   
399  399  58.0  80.0  1.025  0.0  0.0  normal    normal  notpresent   

             ba  ...  pcv    wc   rc  htn   dm  cad appet   pe  ane  \
0    notpresent  ...   44  7800  5.2  yes  yes   no  good   no   no   
1    

**Understanding Data Type and Features**

In [4]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              400 non-null    int64  
 1   age             391 non-null    float64
 2   bp              388 non-null    float64
 3   sg              353 non-null    float64
 4   al              354 non-null    float64
 5   su              351 non-null    float64
 6   rbc             248 non-null    object 
 7   pc              335 non-null    object 
 8   pcc             396 non-null    object 
 9   ba              396 non-null    object 
 10  bgr             356 non-null    float64
 11  bu              381 non-null    float64
 12  sc              383 non-null    float64
 13  sod             313 non-null    float64
 14  pot             312 non-null    float64
 15  hemo            348 non-null    float64
 16  pcv             330 non-null    object 
 17  wc              295 non-null    obj

**Handling Missing Values**

Remove null values

In [5]:
data=data.dropna(how="any")
print(data)

      id   age    bp     sg   al   su       rbc        pc         pcc  \
3      3  48.0  70.0  1.005  4.0  0.0    normal  abnormal     present   
9      9  53.0  90.0  1.020  2.0  0.0  abnormal  abnormal     present   
11    11  63.0  70.0  1.010  3.0  0.0  abnormal  abnormal     present   
14    14  68.0  80.0  1.010  3.0  2.0    normal  abnormal     present   
20    20  61.0  80.0  1.015  2.0  0.0  abnormal  abnormal  notpresent   
..   ...   ...   ...    ...  ...  ...       ...       ...         ...   
395  395  55.0  80.0  1.020  0.0  0.0    normal    normal  notpresent   
396  396  42.0  70.0  1.025  0.0  0.0    normal    normal  notpresent   
397  397  12.0  80.0  1.020  0.0  0.0    normal    normal  notpresent   
398  398  17.0  60.0  1.025  0.0  0.0    normal    normal  notpresent   
399  399  58.0  80.0  1.025  0.0  0.0    normal    normal  notpresent   

             ba  ...  pcv     wc   rc  htn   dm  cad appet   pe  ane  \
3    notpresent  ...   32   6700  3.9  yes   no   n

**Label Encoding** (String values to Numeric values)

In [6]:
data['rbc'] = data['rbc'].map({"abnormal":1,"normal":0})
data['pc'] = data['pc'].map({"abnormal":1,"normal":0})
data['pcc'] = data['pcc'].map({"present":1,"notpresent":0})
data['ba'] = data['ba'].map({"present":1,"notpresent":0})
data['htn'] = data['htn'].map({"yes":1,"no":0})
data['dm'] = data['dm'].map({"yes":1,"no":0})
data['cad'] = data['cad'].map({"yes":1,"no":0})
data['pe'] = data['pe'].map({"yes":1,"no":0})
data['ane'] = data['ane'].map({"yes":1,"no":0})
data['appet'] = data['appet'].map({"poor":1,"good":0})
data['classification'] = data['classification'].map({"ckd":1,"notckd":0})
data['pcv'] = data['pcv'].astype('int')
data['wc'] = data['wc'].astype('int')
data['rc'] = data['rc'].astype('float')
print(data)

      id   age    bp     sg   al   su  rbc  pc  pcc  ba  ...  pcv     wc   rc  \
3      3  48.0  70.0  1.005  4.0  0.0    0   1    1   0  ...   32   6700  3.9   
9      9  53.0  90.0  1.020  2.0  0.0    1   1    1   0  ...   29  12100  3.7   
11    11  63.0  70.0  1.010  3.0  0.0    1   1    1   0  ...   32   4500  3.8   
14    14  68.0  80.0  1.010  3.0  2.0    0   1    1   1  ...   16  11000  2.6   
20    20  61.0  80.0  1.015  2.0  0.0    1   1    0   0  ...   24   9200  3.2   
..   ...   ...   ...    ...  ...  ...  ...  ..  ...  ..  ...  ...    ...  ...   
395  395  55.0  80.0  1.020  0.0  0.0    0   0    0   0  ...   47   6700  4.9   
396  396  42.0  70.0  1.025  0.0  0.0    0   0    0   0  ...   54   7800  6.2   
397  397  12.0  80.0  1.020  0.0  0.0    0   0    0   0  ...   49   6600  5.4   
398  398  17.0  60.0  1.025  0.0  0.0    0   0    0   0  ...   51   7200  5.9   
399  399  58.0  80.0  1.025  0.0  0.0    0   0    0   0  ...   53   6800  6.1   

     htn  dm  cad  appet  p

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

**Splitting Dependent and Independent Variable**

In [7]:
X = data.iloc[:,1:25].values
y = data.iloc[:, 25].values

**Split Train and Test set**

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30,
random_state = 121)#101
print("X train value")
print(X_train)
print("Y train value")
print(y_train)

X train value
[[75.    70.     1.02  ...  0.     0.     0.   ]
 [57.    60.     1.02  ...  0.     0.     0.   ]
 [66.    70.     1.025 ...  0.     0.     0.   ]
 ...
 [58.    80.     1.02  ...  0.     0.     0.   ]
 [73.    80.     1.02  ...  0.     0.     0.   ]
 [46.    60.     1.025 ...  0.     0.     0.   ]]
Y train value
[0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 0 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 1 0 1 0 0 0 0 1 0 1 0 1 0 1 0 0 0
 0 1 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0]


**Model Building**

In [9]:
model = tf.Sequential() 
model.add(tf.layers.Dense(64,input_dim=24,activation='relu'))

**Hidden and Output Layer**

In [10]:
model.add(tf.layers.Dense(128,activation='relu')) 
model.add(tf.layers.Dense(256,activation='relu')) 
model.add(tf.layers.Dense(512,activation='relu')) 
model.add(tf.layers.Dense(1,activation='sigmoid')) 

**Compile Model**

In [11]:
model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])
model.fit(X_train,y_train,epochs=1000)

Epoch 1/1000
4/4 [==============================] - 2s 10ms/step - loss: 223.2395 - accuracy: 0.4182
Epoch 2/1000
4/4 [==============================] - 0s 9ms/step - loss: 216.9276 - accuracy: 0.6909
Epoch 3/1000
4/4 [==============================] - 0s 8ms/step - loss: 103.9522 - accuracy: 0.6545
Epoch 4/1000
4/4 [==============================] - 0s 13ms/step - loss: 45.8573 - accuracy: 0.3455
Epoch 5/1000
4/4 [==============================] - 0s 10ms/step - loss: 37.6950 - accuracy: 0.6909
Epoch 6/1000
4/4 [==============================] - 0s 15ms/step - loss: 16.5266 - accuracy: 0.4364
Epoch 7/1000
4/4 [==============================] - 0s 11ms/step - loss: 10.6547 - accuracy: 0.6909
Epoch 8/1000
4/4 [==============================] - 0s 10ms/step - loss: 11.5420 - accuracy: 0.4091
Epoch 9/1000
4/4 [==============================] - 0s 8ms/step - loss: 10.9178 - accuracy: 0.6909
Epoch 10/1000
4/4 [==============================] - 0s 9ms/step - loss: 8.1410 - accuracy: 0.3727
E

 **Save Model**

In [12]:
# model.save("/content/drive/MyDrive/train.h5")

**Prediction**

In [13]:
ypred=model.predict(X_test)
ypred = ypred.round()
print(ypred)
print(y_test)

2/2 [==============================] - 0s 8ms/step
[[0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]]
[0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0
 1 0 0 1 0 0 0 0 0 1 1]


**Model Evaluation**

In [14]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,ypred)
print(cm)

[[38  1]
 [ 0  9]]


In [15]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,ypred)

0.9791666666666666

In [16]:
import pickle
file = open('/content/drive/MyDrive/ckd.pkl','wb')
pickle.dump(model,file)

In [17]:
from google.colab import files
files.download('/content/drive/MyDrive/ckd.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>